In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hutchinson-Gilford_Progeria_Syndrome"
cohort = "GSE84360"

# Input paths
in_trait_dir = "../../input/GEO/Hutchinson-Gilford_Progeria_Syndrome"
in_cohort_dir = "../../input/GEO/Hutchinson-Gilford_Progeria_Syndrome/GSE84360"

# Output paths
out_data_file = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/GSE84360.csv"
out_gene_data_file = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/gene_data/GSE84360.csv"
out_clinical_data_file = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/clinical_data/GSE84360.csv"
json_path = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Reprogramming Hutchinson-Gilford Progeria Syndrome fibroblasts resets epigenomic landscape in patient-derived induced pluripotent stem cells Jan 01, 2018   pending   None"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['Sex: Male', 'Sex: Female', 'Sex: ?'], 1: ['cell line: HGADFN003', 'cell line: HGMDFN090', 'cell line: HGADFN167', 'cell line: HGFDFN168', 'cell line: AG01972', 'cell line: BJ1', 'cell line: H9'], 2: ['condition: HGPS', 'condition: Normal'], 3: ['cell type: iPSC', 'cell type: Vascular Smooth Muscle', 'cell type: Fibroblast', 'cell type: Embryonic Stem Cell']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Evaluate gene expression data availability
# Given the dataset information and sample characteristics, it appears the data is about cell types from patients with HGPS
# Since we have control and HGPS conditions, this dataset likely contains gene expression data
is_gene_available = True

# 2.1 Data availability and row identification

# For trait (HGPS status)
# Row 2 contains 'condition: HGPS' and 'condition: Normal', indicating disease status
trait_row = 2

# For gender
# Row 0 contains 'Sex: Male', 'Sex: Female', 'Sex: ?', indicating gender information
gender_row = 0

# For age
# No explicit age information is provided in the sample characteristics
age_row = None

# 2.2 Data type conversion functions

def convert_trait(value):
    """
    Convert HGPS trait status to binary (0=Normal, 1=HGPS)
    """
    if not isinstance(value, str):
        return None
    
    # Extract the part after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'hgps':
        return 1
    elif value.lower() == 'normal':
        return 0
    else:
        return None

def convert_gender(value):
    """
    Convert gender to binary (0=Female, 1=Male)
    """
    if not isinstance(value, str):
        return None
    
    # Extract the part after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None

# Age conversion function is defined but not used since age data is not available
def convert_age(value):
    """
    Convert age to continuous value
    """
    if not isinstance(value, str):
        return None
    
    # Extract the part after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 3. Save metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical feature extraction
if trait_row is not None:
    # Extract clinical features using the provided function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM2232606': [1.0, 1.0], 'GSM2232607': [1.0, 1.0], 'GSM2232608': [1.0, 1.0], 'GSM2232609': [1.0, 1.0], 'GSM2232610': [1.0, 1.0], 'GSM2232611': [1.0, 1.0], 'GSM2232612': [1.0, 1.0], 'GSM2232613': [1.0, 1.0], 'GSM2232614': [1.0, 1.0], 'GSM2232615': [0.0, 0.0], 'GSM2232616': [0.0, 0.0], 'GSM2232617': [0.0, 0.0], 'GSM2232618': [0.0, 0.0], 'GSM2232619': [0.0, 0.0], 'GSM2232620': [0.0, 0.0], 'GSM2232621': [0.0, 0.0], 'GSM2232622': [0.0, 0.0], 'GSM2232623': [0.0, 0.0], 'GSM2232624': [1.0, 1.0], 'GSM2232625': [1.0, 1.0], 'GSM2232626': [1.0, 1.0], 'GSM2232627': [1.0, 1.0], 'GSM2232628': [1.0, 1.0], 'GSM2232629': [1.0, 1.0], 'GSM2232630': [1.0, 1.0], 'GSM2232631': [1.0, 1.0], 'GSM2232632': [1.0, 1.0], 'GSM2232633': [0.0, 1.0], 'GSM2232634': [0.0, 1.0], 'GSM2232635': [0.0, 1.0], 'GSM2232636': [0.0, 1.0], 'GSM2232637': [0.0, 1.0], 'GSM2232638': [0.0, 1.0], 'GSM2232639': [0.0, 1.0], 'GSM2232640': [0.0, 1.0], 'GSM2232641': [1.0, 0.0], 'GSM2232642': [1.0, 0.0], 'GSM2232643': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 53617 rows
First 20 gene IDs:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers
# These appear to be probe IDs (numeric identifiers) rather than human gene symbols
# Human gene symbols typically follow naming conventions like BRCA1, TP53, etc.
# These numeric identifiers (16650001, etc.) are likely probe IDs from a microarray platform
# that need to be mapped to official gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # First attempt - use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # We can see this is SNP data, not gene expression data
    print("\nThis dataset contains SNP identifiers (rs numbers), not gene expression probes.")
    print("The data represents genetic variants, not gene expression levels.")
    print("Looking at the columns, we can see Chr and Position information, but no direct gene mapping.")
    
    # Check for genomic position information that could potentially be used for mapping
    print("\nThe data contains genomic position information (Chr, Position) that could be used")
    print("to map SNPs to genes, but this requires external genomic databases.")
    
    # Concluding that this is SNP data, not gene expression data
    print("\nConclusion: This is SNP genotyping data, not gene expression data.")
    print("Traditional gene mapping for expression data is not applicable.")
    print("The initial assessment of is_gene_available=True was incorrect.")
    
    # Update the gene availability flag
    is_gene_available = False
    
    # Update the metadata to reflect that this is not gene expression data
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        note="Dataset contains SNP data, not gene expression data."
    )
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False

Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 2949353 rows

Gene annotation preview (first few rows):
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}

Column names in gene annotation data:
['ID', 'RANGE_STRAND', 'RANGE_START', 'RANGE_END', 'total_probes', 'GB_ACC', 'SPOT_ID', 'RANGE_GB']

This dataset contains SNP identifiers (rs numbers), not gene expression probes.
The data represents genetic variants, not gene expression levels.
Looking at the columns, we can see Chr and Position informat